# **Lab 2: Matrix Factorization**
**Jacob Wahlgren**

# **Abstract**

The goal of lab 2 was to implement sparse matrix-vectoro multiplication, a QR factorization algorithm, and a direct solver of systems of equations. The algorithms were successfully implemented in Python using Numpy and tested with some unit tests.

#**About the code**

The code was written by the author (Jacob Wahlgren), based on a template by Johan Hoffman.

In [2]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)
# Copyright (C) 2021 Jacob Wahlgren (jacobwah@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

In [3]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np
import unittest

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common 
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *
    
#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

A sparse matrix has only $O(n)$ nonzero components. They are common in many applications, and can be handled more efficiently than general matrices by use of special data structures and algorithms. One common structure is CRS (compressed row storage), which is used in this lab to implement the matrix-vector product for sparse matrices.

The goal of matrix factorization is to decompose a matrix into a product of two matrices on some special form. It is useful for computing the inverse of a matrix when the resulting matrices theselves are easy to invert, such as triangular or orthogonal. This is called a direct method of to solve the system of equations represented by the matrix. This lab includes an implementation of QR factorization, i.e. finding an orthogonal matrix $Q$ and a triangular matrix $R$ such that $A=QR$.


# **Method**

Pseudocode for sparse matrix vector product using CRS is given as Algorithm 5.9 in the lecture notes. Unlike the examples in the book, the representation here is zero-indexed to fit better with Python and Numpy array indexing. Consequently the last element of the row pointer vector is the number of non-zero elements (not plus one).

In [4]:
def sparse_matrix_vector_product(x, val, col_idx, row_ptr):
  n = len(x)
  b = np.zeros(n)
  for i in range(n):
    for j in range(row_ptr[i], row_ptr[i+1]):
      b[i] += val[j] * x[col_idx[j]]
  return b

Pseudocode for modified Gram-Schmidt QR factorization is given as Algorithm 5.3 in the lecture notes. Note that we have to make a copy of `a[:,j]` so the original matrix is not overwritten.

In [13]:
def modified_gram_schmidt_iteration(a):
  n = len(a)
  q = np.zeros((n,n))
  r = np.zeros((n,n))
  for j in range(n):
    v = np.copy(a[:,j])
    for i in range(j):
      r[i,j] = np.inner(q[:,i], v)
      v -= r[i,j] * q[:,i]
    r[j,j] = np.linalg.norm(v)
    q[:,j] = v / r[j,j]
  return q, r

The implementation of the direct solver makes use of the QR factorization method above, combined with the backward substitution algorithm from the lecture notes (Algorithm 5.2). First the $Q$ matrix is inverted by taking the transpose, and then $R$ is inverted using backward substitution.

$\displaystyle Ax=QRx=b \implies x = Q^{-1}R^{-1}b = Q^T R^{-1} b$

In [32]:
def backward_substitution(u, b):
  n = len(u)
  x = np.zeros(n)
  x[n-1] = b[n-1] / u[n-1,n-1]
  for i in range(n-2,-1,-1):
    sum = 0
    for j in range(i+1, n):
      sum += u[i,j] * x[j]
    x[i] = (b[i] - sum) / u[i,i]
  return x

def direct_solve(a, b):
  q, r = modified_gram_schmidt_iteration(a)
  rx = q.T @ b
  x = backward_substitution(r, rx)
  return x

# **Results**

The following test cases verify the implementations of sparse matrix-vector product, modified Gram-Schmidt factorization and direct solver.

In [34]:
class SparseTest(unittest.TestCase):
  def test_id(self):
    val = [1, 1, 1]
    col_idx = [0, 1, 2]
    row_ptr = [0, 1, 2, 3]
    np.testing.assert_equal(
        sparse_matrix_vector_product([1, 2, 3], val, col_idx, row_ptr),
        np.array([1, 2, 3]))
    
  def test_full(self):
    val = [1,2,3,4,5,6,7,8,9]
    col_idx = [0,1,2,0,1,2,0,1,2]
    row_ptr = [0,3,6,9]
    x = [10,11,12]
    np.testing.assert_almost_equal(
        sparse_matrix_vector_product(x, val, col_idx, row_ptr),
        np.array([[1,2,3],[4,5,6],[7,8,9]]) @ x
    )

  def test_ex5_5(self):
    # Example 5.5 in the lecture notes
    a = np.array([[3,2,0,2,0,0],
                  [0,2,1,0,0,0],
                  [0,0,1,0,0,0],
                  [0,0,3,2,0,0],
                  [0,0,0,0,1,0],
                  [0,0,0,0,2,3]])
    val = [3,2,2,2,1,1,3,2,1,2,3]
    col_idx = np.array([1,2,4,2,3,3,3,4,5,5,6]) - 1
    row_ptr = np.array([1,4,6,7,9,10,12]) - 1

    x = [1,2,3,4,5,6]

    np.testing.assert_almost_equal(
        sparse_matrix_vector_product(x, val, col_idx, row_ptr),
        a @ x
    )

class QRTest(unittest.TestCase):
  def is_upper_triangular(self, r):
    n = len(r)
    for i in range(n):
      for j in range(i):
        if not np.isclose(r[i,j], 0):
          return False
    return True

  def frobenius_norm(self, q):
    n = len(q)
    x = 0
    for i in range(n):
      for j in range(n):
        x += q[i,j]**2
    return x

  def test_id(self):
    self.check(np.identity(3))
  
  def test_full(self):
    self.check(np.array([[1.,3.,2.],
                        [0.,2.,2.],
                        [1.,-1.,0.]]))

  def check(self, a):
    q, r = modified_gram_schmidt_iteration(a)
    self.assertTrue(self.is_upper_triangular(r))
    np.testing.assert_almost_equal(self.frobenius_norm(q.T @ q - np.identity(3)), 0)
    np.testing.assert_almost_equal(self.frobenius_norm(q @ r - a), 0)

class SolveTest(unittest.TestCase):
  def check(self, a, y):
    b = a @ y
    x = direct_solve(a, b)
    np.testing.assert_almost_equal(np.linalg.norm(x - y), 0)
    np.testing.assert_almost_equal(np.linalg.norm(a @ x - b), 0)

  def test_id(self):
    a = np.identity(3)
    y = np.array([1., 2., 3.])
    self.check(a, y)

  def test_full(self):
    a = np.array([[1.,3.,2.],
                  [0.,2.,2.],
                  [1.,-1.,0.]])
    y = np.array([1.,2.,3.])
    self.check(a, y)
  
unittest.main(argv=[''], verbosity=2, exit=False)

test_full (__main__.QRTest) ... ok
test_id (__main__.QRTest) ... ok
test_full (__main__.SolveTest) ... ok
test_id (__main__.SolveTest) ... ok
test_ex5_5 (__main__.SparseTest) ... ok
test_full (__main__.SparseTest) ... ok
test_id (__main__.SparseTest) ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.014s

OK


# **Discussion**

I had some troubles due to mismatch between the pseduocode in the lecture notes and the Python language. When implementing algorithms specified in pseudocode it is important to keep track of 1-indexed vs 0-indexed code and by-reference vs by-value assignments.

The algorithms implemented in this lab have many applications, but faster and more well tested implementations are found in libraries.